In [1]:
import re
import glob
import json
from bs4 import BeautifulSoup
from unidecode import unidecode

### Clean Json File

#### all sections tag included

In [14]:
def cleanup(json_toload):
    js= json.load(open(json_toload))
    d=[]
    for j in range(len(js['rows'])):
        data_dict={}
        title=js["rows"][j]["title"]
        cont="".join(js["rows"][j]["content"])
        time=js["rows"][j]['lastupdated']['iso8601']
        section=js["rows"][j]['sections']
        content=unidecode(cont)
        soup = BeautifulSoup(content, 'lxml')
        
        [tagstring.extract() for tagstring in soup('script')] ### extracts all content not between balance <script> tags
        [tagstring.extract() for tagstring in soup('table')] ### extracts all content not between balance <table> tags
        [tagstring.extract() for tagstring in soup('iframe')] ### extracts all content not between balance <iframe> tags
        [tagstring.extract() for tagstring in soup('blockquote')] ### extracts all content not between balance <iblockquote> tags
        [tagstring.extract() for tagstring in soup('ol')] ### extracts all content not between balance <ol> tags
        [tagstring.extract() for tagstring in soup('ul')] ### extracts all content not between balance <ul> tags
        [tagstring.extract() for tagstring in soup('h1')] ### extracts all content not between balance <h1> tags
        [tagstring.extract() for tagstring in soup('h2')] ### extracts all content not between balance <h2> tags
        [tagstring.extract() for tagstring in soup('h3')] ### extracts all content not between balance <h3> tags
        [tagstring.extract() for tagstring in soup('h4')] ### extracts all content not between balance <h4> tags
        [tagstring.extract() for tagstring in soup('h5')] ### extracts all content not between balance <h5> tags
        [tagstring.extract() for tagstring in soup('img')] ### extracts all content not between balance <img> tags
        filtered_content = soup.find_all('p')
        moment = ""
        for wrapped_paragraph in filtered_content:
            moment+= str(wrapped_paragraph.wrap)+" "

            content = re.sub(r"<bound.*?>"," ",content).strip() ###some article begin with this
            content= re.sub(r'/^(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?$/', ' ', content,re.MULTILINE)
            content = re.sub(r"<\!--.*"," ",content,re.DOTALL)
            content = re.sub(r"\.\s{,2}\.\s",". ",content,re.DOTALL)
            content = re.sub(r"\[.*?\]"," ",content,re.DOTALL)
            content = re.sub(r"\<span.*?</span\>"," ",content,re.DOTALL)
            content = content.replace('<div itemprop="reviewBody">'," ") ###some article begin with this
            content = content.replace("@"," ")
            content = content.replace("|"," ")
            content = content.replace("/",",")
            content = content.replace("(",",")
            content = content.replace(")"," ")
            content = content.replace("---",". ")
            content = content.replace("$"," ")
            content = re.sub(r"<.*?>"," " ,content,1000,re.DOTALL)
            data_dict=dict([('title',title),('time',time),('section',section),('content',content)])
        d.append(data_dict)
    return d

In [15]:
data=[]
for i in glob.glob(r'C:\Users\yueyu\OneDrive\桌面\Corpus\*.json'):
    data.append(cleanup(i))

In [16]:
len(data)

741

In [46]:
data[33][1]['section'][0]

'business/consumer'

In [20]:
import pandas as pd
combine=[]
for i in data:
    for j in i:
        combine.append(j)

In [22]:
d=pd.DataFrame(combine)

In [25]:
d.to_csv('clean_article_in_dict.csv',index=False)

In [3]:
import pandas as pd
m=pd.read_csv('clean_article_in_dict.csv')

### Drop dulicates

In [6]:
data=m.drop_duplicates()

In [7]:
data.to_csv('noDup_data.csv',index=False)